In [1]:
import pandas as pd
import numpy as np

# Load original dataset
df = pd.read_excel("data/shipment_dataset_10000.xlsx")
df_anom = df.copy()

print("Adding simple anomalies...")

# 1. Add some missing values
df_anom.loc[df_anom.sample(20).index, 'supplier_rating'] = np.nan
df_anom.loc[df_anom.sample(20).index, 'order_quantity'] = np.nan

# 2. Add duplicate rows
df_anom = pd.concat([df_anom, df_anom.sample(10)], ignore_index=True)

# 3. Add datatype mismatch
df_anom.loc[df_anom.sample(10).index, 'order_quantity'] = "error_value"

# 4. Add outliers
df_anom.loc[df_anom.sample(10).index, 'shipping_distance_km'] = 99999

# 5. Add wrong dates (delivery before order)
df_anom.loc[df_anom.sample(10).index, 'actual_delivery_date'] = (
    df_anom['order_date'] - pd.Timedelta(days=3)
)

# Save file
df_anom.to_excel("data/simple_anomalies_dataset.xlsx", index=False)

print("Anomaly file created: simple_anomalies_dataset.xlsx")


Adding simple anomalies...


C:\Users\prana\AppData\Local\Temp\ipykernel_21712\3316039859.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'error_value' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_anom.loc[df_anom.sample(10).index, 'order_quantity'] = "error_value"


Anomaly file created: simple_anomalies_dataset.xlsx


In [3]:
import pandas as pd
import numpy as np

print("Cleaning anomalies...")

# Load anomalous dataset
df = pd.read_excel("data/simple_anomalies_dataset.xlsx")
df_clean = df.copy()

# 1. Remove duplicates
df_clean = df_clean.drop_duplicates()

# 2. Fix datatype mismatches
df_clean['order_quantity'] = pd.to_numeric(df_clean['order_quantity'], errors='coerce')

# 3. Fill missing values
df_clean['supplier_rating'] = df_clean['supplier_rating'].fillna(df_clean['supplier_rating'].median())
df_clean['order_quantity'] = df_clean['order_quantity'].fillna(df_clean['order_quantity'].median())

# 4. Fix wrong dates
df_clean = df_clean[df_clean['actual_delivery_date'] >= df_clean['order_date']]

# 5. Treat outliers in shipping distance
upper_limit = df_clean['shipping_distance_km'].quantile(0.95)
df_clean['shipping_distance_km'] = np.where(
    df_clean['shipping_distance_km'] > upper_limit,
    upper_limit,
    df_clean['shipping_distance_km']
)

# Save cleaned dataset
df_clean.to_excel("data/simple_cleaned_dataset.xlsx", index=False)

print("Cleaned file created: simple_cleaned_dataset.xlsx")


Cleaning anomalies...
Cleaned file created: simple_cleaned_dataset.xlsx


Step 1 : Load the 3 datasets

In [ ]:
import pandas as pd

df_original = pd.read_excel("data/shipment_dataset_10000.xlsx")
df_anomalies = pd.read_excel("data/simple_anomalies_dataset.xlsx")
df_cleaned = pd.read_excel("data/simple_cleaned_dataset.xlsx")

df_original.head(), df_anomalies.head(), df_cleaned.head()


FileNotFoundError: [Errno 2] No such file or directory: 'shipment_dataset_10000.xlsx'

step 2 :Compare shapes (rows × columns)

In [4]:
print("Original:", df_original.shape)
print("With Anomalies:", df_anomalies.shape)
print("Cleaned:", df_cleaned.shape)


Original: (10000, 19)
With Anomalies: (10010, 19)
Cleaned: (9990, 19)


step 3 : Compare duplicates

In [5]:
print("Duplicates in Original:", df_original.duplicated().sum())
print("Duplicates After Adding Anomalies:", df_anomalies.duplicated().sum())
print("Duplicates After Cleaning:", df_cleaned.duplicated().sum())


Duplicates in Original: 0
Duplicates After Adding Anomalies: 10
Duplicates After Cleaning: 0


step 4 : Compare missing values

In [6]:
print("Missing values - ORIGINAL")
display(df_original.isna().sum())

print("Missing values - WITH ANOMALIES")
display(df_anomalies.isna().sum())

print("Missing values - CLEANED")
display(df_cleaned.isna().sum())


Missing values - ORIGINAL


order_id                     0
supplier_id                  0
supplier_rating              0
supplier_lead_time           0
order_date                   0
promised_delivery_date       0
actual_delivery_date         0
shipment_mode                0
shipping_distance_km         0
order_quantity               0
unit_price                   0
total_order_value            0
weather_condition            0
region                       0
holiday_period               0
previous_on_time_rate        0
carrier_name                 0
delayed_reason_code       1998
on_time_delivery             0
dtype: int64

Missing values - WITH ANOMALIES


order_id                     0
supplier_id                  0
supplier_rating             20
supplier_lead_time           0
order_date                   0
promised_delivery_date       0
actual_delivery_date         0
shipment_mode                0
shipping_distance_km         0
order_quantity              20
unit_price                   0
total_order_value            0
weather_condition            0
region                       0
holiday_period               0
previous_on_time_rate        0
carrier_name                 0
delayed_reason_code       1999
on_time_delivery             0
dtype: int64

Missing values - CLEANED


order_id                     0
supplier_id                  0
supplier_rating              0
supplier_lead_time           0
order_date                   0
promised_delivery_date       0
actual_delivery_date         0
shipment_mode                0
shipping_distance_km         0
order_quantity               0
unit_price                   0
total_order_value            0
weather_condition            0
region                       0
holiday_period               0
previous_on_time_rate        0
carrier_name                 0
delayed_reason_code       1995
on_time_delivery             0
dtype: int64

step 5 : Show datatype mismatches fixed

In [7]:
df_anomalies[df_anomalies['order_quantity'] == 'error_value'].head()


,order_id,supplier_id,supplier_rating,supplier_lead_time,order_date,promised_delivery_date,actual_delivery_date,shipment_mode,shipping_distance_km,order_quantity,unit_price,total_order_value,weather_condition,region,holiday_period,previous_on_time_rate,carrier_name,delayed_reason_code,on_time_delivery
1291,1292,9014,4.8,9,2024-11-13,2024-11-22,2024-11-22,Sea,428,error_value,3354.59,167729.50,Storm,West,No,82.6,EcomExpress,Customs,1
1341,1342,1299,4.8,9,2024-07-22,2024-07-31,2024-08-02,Air,1241,error_value,3508.33,336799.68,Storm,West,Yes,97.0,EcomExpress,Weather,0
1934,1935,4690,3.3,3,2024-01-08,2024-01-11,2024-01-10,Road,871,error_value,4484.84,4484.84,Rainy,West,Yes,93.5,EcomExpress,NaN,1
2557,2558,7092,4.6,7,2024-08-01,2024-08-08,2024-08-10,Sea,515,error_value,4274.39,175249.99,Clear,West,Yes,88.3,EcomExpress,Traffic,0
3721,3722,4517,3.0,2,2024-08-28,2024-08-30,2024-09-04,Air,1178,error_value,673.86,56604.24,Rainy,East,No,91.5,BlueDart,Operational,0


In [8]:
df_cleaned[df_cleaned['order_quantity'] == 'error_value']


,order_id,supplier_id,supplier_rating,supplier_lead_time,order_date,promised_delivery_date,actual_delivery_date,shipment_mode,shipping_distance_km,order_quantity,unit_price,total_order_value,weather_condition,region,holiday_period,previous_on_time_rate,carrier_name,delayed_reason_code,on_time_delivery


Step 6 : Compare outliers

In [9]:
df_anomalies['shipping_distance_km'].describe()


count    10010.000000
mean       859.739161
std       3164.587218
min         10.000000
25%        391.000000
50%        760.000000
75%       1137.000000
max      99999.000000
Name: shipping_distance_km, dtype: float64

In [10]:
df_cleaned['shipping_distance_km'].describe()


count    9990.000000
mean      759.731532
std       427.889853
min        10.000000
25%       391.250000
50%       760.000000
75%      1137.000000
max      1430.000000
Name: shipping_distance_km, dtype: float64

step 7 :  Compare wrong dates

In [11]:
bad_dates = df_anomalies[df_anomalies['actual_delivery_date'] < df_anomalies['order_date']]
bad_dates.head()
len(bad_dates)


10

In [12]:
bad_dates_clean = df_cleaned[df_cleaned['actual_delivery_date'] < df_cleaned['order_date']]
len(bad_dates_clean)


0

step 8 : BEFORE/AFTER SUMMARY TABLE

In [13]:
summary = pd.DataFrame({
    "Original": [
        df_original.duplicated().sum(),
        df_original.isna().sum().sum(),
    ],
    "After Adding Anomalies": [
        df_anomalies.duplicated().sum(),
        df_anomalies.isna().sum().sum(),
    ],
    "After Cleaning": [
        df_cleaned.duplicated().sum(),
        df_cleaned.isna().sum().sum(),
    ]
}, index=["Total Duplicates", "Total Missing Values"])

summary


,Original,After Adding Anomalies,After Cleaning
Total Duplicates,0,10,0
Total Missing Values,1998,2039,1995
